In [2]:
import matplotlib.pyplot as plt                        
%matplotlib inline    

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms

import os
import re
from pathlib import Path
from glob import glob
import cv2 
import numpy as np
import pandas as pd


In [26]:
mids = []
tmp_fid = 0
tmp_mid = 0
mid_object = None
df = pd.read_csv("v0.1.2/lists/imagelist.csv", index_col=None, header=0)
print(len(df))
df["NAME"] = ""
df["GENDER"] =""
df["CHILD"] = 0
df["SIBLING"] = 0
df["GRANDCHILD"] = 0
df["PARENT"] = 0
df["SPOUSE"] = 0
df["GRANDPARENT"] = 0
df["GREATGRANDCHILD"] = 0
df["GREATGRANDPARENT"] = 0

#EXTRACT ALL RELATIONSHIP FOR EACH FOLDER
fids = df.FID.unique().tolist()
fid_mid_ix = df.groupby(['FID', 'MID']).size().reset_index(name='Freq')
columns = []
for fid in fids:
    fid_df = pd.read_csv("v0.1.2/FIDs/"+fid+"/mid.csv", index_col=None, header=0)
    mids_fid = fid_mid_ix[fid_mid_ix["FID"] == fid]
    for index, mid_row in mids_fid.iterrows():
        mid_value = int(re.findall('\d+', mid_row.MID )[0])
        mid_dtl = fid_df.loc[fid_df["MID"] == mid_value]
        #print(mid_dtl)
        for col in mid_dtl.columns: 
            if col not in ["MID","Name","Gender",mid_value]:
                columns.append(col)
        if(len(mid_dtl)==0): ## no information was provided in the mapping in the mid.csv
            df.drop(df[(df.FID == fid) & (df.MID == "MID"+str(mid_value))].index,inplace=True)
            print("drop",fid,mid_value," because they don't have any record in the mid.csv")
        else:
            for col in columns:
                if (mid_dtl[col].values[0] ==1): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["CHILD"]] = 1
                elif (mid_dtl[col].values[0] ==2): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["SIBLING"]] = 1
                elif (mid_dtl[col].values[0] ==3): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["GRANDCHILD"]] = 1
                elif (mid_dtl[col].values[0] ==4): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["PARENT"]] = 1
                elif (mid_dtl[col].values[0] ==5): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["SPOUSE"]] = 1
                elif (mid_dtl[col].values[0] ==6): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["GRANDPARENT"]] = 1
                elif (mid_dtl[col].values[0] ==7): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["GREATGRANDCHILD"]] = 1
                elif (mid_dtl[col].values[0] ==8): #some people has no relation like son-in-law
                     df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["GREATGRANDPARENT"]] = 1
                else:
                    x = 1
                    #print("NOT APPLICAPLE",fid,mid_value,mid_dtl[col].values[0])
            df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["NAME"]] = mid_dtl["Name"].values[0]
            df.loc[(df.FID == fid) & (df.MID == "MID"+str(mid_value)),["GENDER"]] = mid_dtl["Gender"].values[0]
        columns.clear()
df.head()
print(len(df))

26643
drop F0396 3  because they don't have any record in the mid.csv
drop F0396 4  because they don't have any record in the mid.csv
drop F0396 5  because they don't have any record in the mid.csv
drop F0601 43  because they don't have any record in the mid.csv
drop F0601 44  because they don't have any record in the mid.csv
drop F0601 45  because they don't have any record in the mid.csv
drop F0757 4  because they don't have any record in the mid.csv
drop F0757 5  because they don't have any record in the mid.csv
drop F0757 6  because they don't have any record in the mid.csv
drop F0757 7  because they don't have any record in the mid.csv
drop F0757 8  because they don't have any record in the mid.csv
drop F0757 9  because they don't have any record in the mid.csv
26620


In [25]:
df.loc[df["FID"] == "F0001"].head(100)

,ID,file,FID,MID,NAME,GENDER,CHILD,SIBLING,GRANDCHILD,PARENT,SPOUSE,GRANDPARENT,GREATGRANDCHILD,GREATGRANDPARENT
0,0,F0001/MID1/P00001_face0.jpg,F0001,MID1,madmoud,m,0,0,0,4,5,0,0,0
1,0,F0001/MID1/P00002_face0.jpg,F0001,MID1,madmoud,m,0,0,0,4,5,0,0,0
2,0,F0001/MID1/P00003_face0.jpg,F0001,MID1,madmoud,m,0,0,0,4,5,0,0,0
3,0,F0001/MID1/P00004_face1.jpg,F0001,MID1,madmoud,m,0,0,0,4,5,0,0,0
4,0,F0001/MID1/P00007_face1.jpg,F0001,MID1,madmoud,m,0,0,0,4,5,0,0,0
5,0,F0001/MID1/P00008_face4.jpg,F0001,MID1,madmoud,m,0,0,0,4,5,0,0,0
6,1,F0001/MID2/P00001_face1.jpg,F0001,MID2,amina,f,0,0,0,4,5,0,0,0
7,1,F0001/MID2/P00002_face1.jpg,F0001,MID2,amina,f,0,0,0,4,5,0,0,0
8,1,F0001/MID2/P00003_face3.jpg,F0001,MID2,amina,f,0,0,0,4,5,0,0,0
9,1,F0001/MID2/P00004_face3.jpg,F0001,MID2,amina,f,0,0,0,4,5,0,0,0


In [31]:
parents = df.loc[df["PARENT"]==1]
parents.head(100)

,ID,file,FID,MID,NAME,GENDER,CHILD,SIBLING,GRANDCHILD,PARENT,SPOUSE,GRANDPARENT,GREATGRANDCHILD,GREATGRANDPARENT
0,0,F0001/MID1/P00001_face0.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
1,0,F0001/MID1/P00002_face0.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
2,0,F0001/MID1/P00003_face0.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
3,0,F0001/MID1/P00004_face1.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
4,0,F0001/MID1/P00007_face1.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,22,F0006/MID2/P00066_face1.jpg,F0006,MID2,leslie,f,0,0,0,1,1,0,0,0
152,22,F0006/MID2/P00067_face4.jpg,F0006,MID2,leslie,f,0,0,0,1,1,0,0,0
153,22,F0006/MID2/P00069_face2.jpg,F0006,MID2,leslie,f,0,0,0,1,1,0,0,0
154,22,F0006/MID2/P00070_face0.jpg,F0006,MID2,leslie,f,0,0,0,1,1,0,0,0


In [33]:
fathers = df.loc[(df["PARENT"]==1) & (df["GENDER"] == "m")]
fathers.head(10)

,ID,file,FID,MID,NAME,GENDER,CHILD,SIBLING,GRANDCHILD,PARENT,SPOUSE,GRANDPARENT,GREATGRANDCHILD,GREATGRANDPARENT
0,0,F0001/MID1/P00001_face0.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
1,0,F0001/MID1/P00002_face0.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
2,0,F0001/MID1/P00003_face0.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
3,0,F0001/MID1/P00004_face1.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
4,0,F0001/MID1/P00007_face1.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
5,0,F0001/MID1/P00008_face4.jpg,F0001,MID1,madmoud,m,0,0,0,1,1,0,0,0
15,4,F0002/MID1/P00009_face0.jpg,F0002,MID1,greg,m,0,0,0,1,1,0,0,0
16,4,F0002/MID1/P00010_face3.jpg,F0002,MID1,greg,m,0,0,0,1,1,0,0,0
17,4,F0002/MID1/P00011_face2.jpg,F0002,MID1,greg,m,0,0,0,1,1,0,0,0
18,4,F0002/MID1/P00012_face1.jpg,F0002,MID1,greg,m,0,0,0,1,1,0,0,0


In [34]:
childs = df.loc[df["CHILD"]==1]
childs.head()

,ID,file,FID,MID,NAME,GENDER,CHILD,SIBLING,GRANDCHILD,PARENT,SPOUSE,GRANDPARENT,GREATGRANDCHILD,GREATGRANDPARENT
11,2,F0001/MID3/P00006_face0.jpg,F0001,MID3,tareq,m,1,1,0,0,0,0,0,0
12,3,F0001/MID4/P00002_face3.jpg,F0001,MID4,yasser,m,1,1,0,0,0,0,0,0
13,3,F0001/MID4/P00006_face1.jpg,F0001,MID4,yasser,m,1,1,0,0,0,0,0,0
14,3,F0001/MID4/P00008_face3.jpg,F0001,MID4,yasser,m,1,1,0,0,0,0,0,0
34,6,F0002/MID3/P00009_face2.jpg,F0002,MID3,audrey,f,1,0,0,0,0,0,0,0
